In [35]:
import os
from pathlib import Path
import sys
curdir = Path(os.getcwd())
sys.path.append(str(curdir.parent.absolute()))
from collections import Counter
import pandas as pd
from src.utils.data import read_fasta
from src.data.datasets import ProteinDataset
import numpy as np
from src.utils.data import read_pickle, save_to_pickle
from src.utils.evaluation import metrics_per_label_df
from torchmetrics.classification import AveragePrecision,Specificity
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from src.utils.losses import FocalLoss
from src.utils.evaluation import EvalMetrics


In [36]:
read_fasta("../data/zero_shot/inference.fasta")

[('MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLEGCPTNGPKIPSIATGMVGALLLLLVVALGIGLFMRRRHIVRKRTLRRLLQERELVEPLTPSGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNF

In [37]:

logits = pd.read_parquet("../outputs/results/None_logits_inference.parquet")
labels = pd.read_parquet("../outputs/results/None_labels_inference.parquet")

In [39]:
torch.sigmoid(torch.tensor(logits.values))[0]

tensor([9.9572e-01, 1.6929e-06, 1.3512e-01, 6.7630e-01, 8.2332e-08, 9.9898e-01,
        8.9010e-01, 9.9915e-01, 5.2966e-06])

In [7]:
labels

,N1,N2,P1,P2,P3,P4,P5
0,1,1,1,1,1,1,1


In [14]:
annotations.loc['GO:0140096']

label              Catalytic activity that acts to modify a protein.
name                         catalytic activity, acting on a protein
synonym_exact                                                     []
synonym_narrow                                                    []
synonym_related                                                   []
synonym_broad                                                     []
is_obsolete                                                      NaN
Name: GO:0140096, dtype: object

In [22]:
'GO:0140096' in new

False

In [26]:
logits.loc['P00533'].sort_values() #['GO:0005515']

GO:0000510   -16.125000
GO:0140674   -16.125000
GO:0140679   -16.125000
GO:0140680   -16.125000
GO:0140681   -16.125000
                ...    
GO:0106264    -0.722168
GO:0140431    -0.126099
GO:0120274     0.736816
GO:0106310     1.991211
GO:0003674    15.945312
Name: P00533, Length: 731, dtype: float32

In [33]:
labels.loc['P00533',logits.loc['P00533'].sort_values().index]

GO:0000510    0
GO:0140674    0
GO:0140679    0
GO:0140680    0
GO:0140681    0
             ..
GO:0106264    0
GO:0140431    0
GO:0120274    0
GO:0106310    0
GO:0003674    1
Name: P00533, Length: 731, dtype: int64

In [15]:
#Read Val Datas
#final_validation_labels_fl_mlpscale2_projection_head_2layers

logits = pd.read_parquet("../outputs/results/test_1_logits_all_swissprot.parquet")
labels = pd.read_parquet("../outputs/results/test_1_labels_all_swissprot.parquet")

# logits = pd.read_parquet("../outputs/results/test_1_logits_all_swissprot_normal.parquet")
# labels = pd.read_parquet("../outputs/results/test_1_labels_all_swissprot_normal.parquet")

# logits = pd.read_parquet("../outputs/results/test_1_logits_all_swissprot_normal_v2.parquet")
# labels = pd.read_parquet("../outputs/results/test_1_labels_all_swissprot_normal_v2.parquet")


annotations = pd.read_pickle('../data/annotations/go_annotations_2019_07_01_updated.pkl')
annotations_old = pd.read_pickle('../data/annotations/go_annotations_2019_07_01.pkl')


In [20]:
unseen_test = read_fasta('../data/zero_shot/SwissProt_2023_unseen_sequences_and_labels.fasta')
full_go = read_fasta('../data/swissprot/proteinfer_splits/random/full_GO.fasta')

unseen_terms = set([j for i in unseen_test for j in i[-1]])
unseen_seqs = set([i[0] for i in unseen_test])
unseen_seq_ids = set([i[1] for i in unseen_test])

full_go_terms = set([j for i in full_go for j in i[-1]])
new =  set(annotations.index) - set(annotations_old.index)

In [4]:
print('New annotations from new terms:', len(unseen_terms))
print('New annotations:', len(new))
print('New predicted labels all swissprot',len(set(labels.columns)))


New annotations from new terms: 228
New annotations: 1260
New predicted labels all swissprot 731


In [5]:
all_sp = read_fasta('../data/swissprot/swissprot_2023.fasta')
id2seq = {id:seq for seq,id,_ in all_sp}

for df in [labels,logits]:
    df.drop(['GO:0003674'],axis=1,inplace=True) #Removing dummy GO Annotation
    df['sequence'] = list(df.index.map(id2seq))

    #Identify sequences based on proteinfer's split
    seq2split = {
    **{seq:'train' for seq,_,_ in read_fasta('../data/swissprot/proteinfer_splits/random/train_GO.fasta')},
    **{seq:'val' for seq,_,_ in read_fasta('../data/swissprot/proteinfer_splits/random/dev_GO.fasta')},
    **{seq:'test' for seq,_,_ in read_fasta('../data/swissprot/proteinfer_splits/random/test_GO.fasta')}
    }

    df['split'] = df['sequence'].map(seq2split).fillna('new')

    df.set_index(['sequence','split'],append=True,inplace=True)

There are some proteins in proteinfer dataset that don't appear in all swissprot data. Less than 0.3% of proteinfer sequences. It's just proteins that have "INLL" appended at the end

In [6]:
from collections import defaultdict
del_seqs = set([i[0] for i in full_go]) - set(id2seq.values())
full_go_seq2ids = defaultdict(list)
for seq,id,_ in full_go:
    full_go_seq2ids[seq].append(id)



In [7]:
len(del_seqs)*100/len(full_go)

0.2965899806164097

In [8]:
[full_go_seq2ids[i] for i in del_seqs][:10]

[['Q9GLP6'],
 ['Q11098'],
 ['P17019'],
 ['Q90632'],
 ['Q96SE0'],
 ['Q5H9K5'],
 ['P63034'],
 ['Q96ME1'],
 ['A8WRP9'],
 ['Q09M05']]

unseen_test

In [9]:
def fasta_list_to_df(fasta_list):
    df = pd.DataFrame([(seq,id," ".join(labs)) for seq,id,labs in fasta_list],columns=['sequence','id','labels'])
    return df

In [10]:
unseen_test_df = fasta_list_to_df(unseen_test)
all_sp_df = fasta_list_to_df(all_sp)

all_sp_df['labels'] = all_sp_df['labels'].apply(lambda x: sorted(x.replace('GO:0003674','').replace('  ',' ').strip().split(' ')))
unseen_test_df['labels'] = unseen_test_df['labels'].apply(lambda x: sorted(x.strip().split(' ')))


Performance

In [11]:
from torcheval.metrics import MultilabelAUPRC, BinaryAUPRC
eval_metrics = EvalMetrics(device='cpu')


for split in [
              #'val',
              'test',
              #'new'
              ]:
    
    label_mask = labels.query('split == @split').sum(axis=0)>=0
    label_mask = list(label_mask[label_mask].index)

    sequence_mask = labels.query('split == @split').sum(axis=1)>=0
    sequence_mask = list(sequence_mask[sequence_mask].index)


    mAP_micro = BinaryAUPRC(device='cpu')
    mAP_macro = MultilabelAUPRC(device='cpu',
                                num_labels=len(label_mask))
    metrics=eval_metrics.get_metric_collection_with_regex(pattern="(f1_m.*)|(precision_macro)|(recall_macro)",
                                                                    threshold=0.3,
                                                            num_labels=len(label_mask)
                                                            )
    
    probabilities = torch.sigmoid(torch.tensor(logits.query('split == @split').loc[sequence_mask][label_mask].values))
    y = torch.tensor(labels.query('split == @split').loc[sequence_mask][label_mask].values)


    print(mAP_micro.update(probabilities.flatten(), y.flatten()).compute(),mAP_macro.update(probabilities, y).compute())
    print(metrics(probabilities, y))
    metrics.reset()
    print(metrics(probabilities, 1-y))

tensor(0.0117) tensor(0.0619)
{'f1_macro': tensor(0.0393), 'f1_micro': tensor(0.0251), 'precision_macro': tensor(0.0341), 'recall_macro': tensor(0.0925)}
{'f1_macro': tensor(0.0020), 'f1_micro': tensor(0.0022), 'precision_macro': tensor(0.7317), 'recall_macro': tensor(0.0011)}


In [61]:
probabilities_df = pd.DataFrame(probabilities,
                                columns = logits.query('split == @split').loc[sequence_mask][label_mask].columns,
                                index = logits.query('split == @split').loc[sequence_mask][label_mask].index)

In [77]:
m=(probabilities_df>0.9)&(labels.query('split == @split').loc[sequence_mask][label_mask]==0)

In [26]:
from sklearn.metrics import confusion_matrix

In [40]:
torch.mean(y[probabilities>0.99].float())

tensor(0.0352)

In [60]:
torch.where((probabilities>0.99)&(y==0))

(tensor([  225,   840,  1002,  2066,  2071,  4416,  5082,  5286,  5286,  9112,
          9113,  9653,  9653,  9783,  9784,  9785,  9953,  9958,  9962, 10231,
         10247, 11231, 11474, 11476, 11477, 11481, 11487, 11489, 11493, 11499,
         11503, 11504, 11505, 11508, 11510, 12198, 12826, 13434, 13564, 13583,
         16123, 16123, 16900, 16942, 16942, 16945, 16945, 16946, 16946, 17633,
         17922, 18176, 18940, 18941, 19515, 19516, 20691, 21503, 21650, 23392,
         23396, 23396, 24572, 24573, 24574, 24575, 25507, 25507, 25881, 25886,
         25887, 26006, 26006, 26007, 26008, 26009, 26010, 26010, 26011, 26012,
         26013, 26014, 26015, 26016, 26017, 26018, 26019, 26020, 26748, 27477,
         27480, 27482, 27491, 27505, 27544, 27545, 28216, 28216, 28230, 28390,
         28392, 28394, 28395, 28396, 28794, 28795, 29020, 29097, 29456, 29457,
         29458, 29459, 29467, 29468, 29469, 29470, 29471, 29472, 29473, 29474,
         29475, 29478, 29480, 29492, 29493, 29494, 2

In [49]:
y[probabilities>0.].shape

torch.Size([19598])

In [53]:
y.sum()

tensor(1804)

In [51]:
51683*730

37728590

In [54]:
1804/37728590

4.7815197970557606e-05

In [52]:
19598/37728590

0.0005194469234074213

In [55]:
tn,fp,fn,tp = confusion_matrix(y.flatten().numpy(),(probabilities.flatten().numpy()>.3)*1).ravel()

In [56]:
tn,fp,fn,tp

(37685570, 41216, 1257, 547)

In [44]:
tn,fp,fn,tp

(37726428, 358, 1800, 4)

In [24]:
from torcheval.metrics import MultilabelAUPRC, BinaryAUPRC
eval_metrics = EvalMetrics(device='cpu')


for split in [
              #'val',
              'test',
              #'new'
              ]:
    
    label_mask = labels.query('split == @split').sum(axis=0)>0
    label_mask = list(label_mask[label_mask].index)

    sequence_mask = labels.query('split == @split').sum(axis=1)>=0
    sequence_mask = list(sequence_mask[sequence_mask].index)


    mAP_micro = BinaryAUPRC(device='cpu')
    mAP_macro = MultilabelAUPRC(device='cpu',
                                num_labels=len(label_mask))
    metrics=eval_metrics.get_metric_collection_with_regex(pattern="(f1_m.*)|(precision_macro)|(recall_macro)",
                                                                    threshold=0.3,
                                                            num_labels=len(label_mask)
                                                            )
    
    probabilities = torch.sigmoid(torch.tensor(logits.query('split == @split').loc[sequence_mask][label_mask].values))
    y = torch.tensor(labels.query('split == @split').loc[sequence_mask][label_mask].values)


    print(mAP_micro.update(probabilities.flatten(), y.flatten()).compute(),mAP_macro.update(probabilities, y).compute())
    print(metrics(probabilities, y))
    metrics.reset()
    print(metrics(probabilities, 1-y))

tensor(0.0477) tensor(0.1429)
{'f1_macro': tensor(0.0938), 'f1_micro': tensor(0.0763), 'precision_macro': tensor(0.0823), 'recall_macro': tensor(0.2208)}
{'f1_macro': tensor(0.0023), 'f1_micro': tensor(0.0024), 'precision_macro': tensor(0.6941), 'recall_macro': tensor(0.0012)}


In [1]:
labels.query('split == @split').loc[sequence_mask][label_mask].shape

NameError: name 'labels' is not defined

In [13]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

In [ ]:
seq2split

In [14]:
test_new_labels = []
test_new_labels_ = []
for i in all_sp:
    if i[0] in seq2split:
        if seq2split[i[0]]=='test':
            record = SeqRecord(Seq(i[0]),id=i[1],description=" ".join(i[2]))
            test_new_labels.append(record)
            test_new_labels_.append(i)


In [15]:
len(test_new_labels)

51687

In [25]:
seqs = set([i[0] for i in test_new_labels_])
labs=labels.query('sequence in @seqs')
lags=logits.query('sequence in @seqs')

In [52]:
logs = pd.read_parquet("../outputs/results/test_1_logits_zero_shot_pinf_test.parquet").drop('GO:0003674',axis=1)
labs = pd.read_parquet("../outputs/results/test_1_labels_zero_shot_pinf_test.parquet").drop('GO:0003674',axis=1)
mAP_micro = BinaryAUPRC(device='cpu')
mAP_macro = MultilabelAUPRC(device='cpu',
                            num_labels=labs.shape[-1])

probabilities = torch.sigmoid(torch.tensor(logs.values))
y = torch.tensor(labs.values)

print(mAP_micro.update(probabilities.flatten(), y.flatten()).compute(),mAP_macro.update(probabilities, y).compute())

tensor(0.0478) tensor(0.1399)


In [37]:
labs

,GO:0000510,GO:0000511,GO:0000514,GO:0000515,GO:0003674,GO:0062152,GO:0062153,GO:0062156,GO:0062157,GO:0062173,...,GO:0160047,GO:0160049,GO:0160063,GO:0160064,GO:0160072,GO:0160075,GO:0170014,GO:0180003,GO:0180010,GO:0180019
Q6GZX1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q6GZW5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q197E7,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q6GZQ0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q6GZN7,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q6PD29,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q96K75,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q76KX8,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q6NX49,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:

label_mask = labels.query('split == @split').sum(axis=0)>0
label_mask = list(label_mask[label_mask].index)

sequence_mask = labels.query('split == @split').sum(axis=1)>0
sequence_mask = list(sequence_mask[sequence_mask].index)

In [66]:
labels.loc[sequence_mask][label_mask]

,,,GO:0000514,GO:0000515,GO:0062136,GO:0062142,GO:0062143,GO:0062144,GO:0062145,GO:0062146,GO:0062147,GO:0062148,...,GO:0160063,GO:0160064,GO:0160076,GO:0160082,GO:0170011,GO:0170014,GO:0180000,GO:0180011,GO:0180015,GO:0180017
,sequence,split,,,,,,,,,,,,,,,,,,,,,
G2X4G0,MVSFTSLLAAFSVVSGVLTSPIAVVPEVNTALAKRTPSSTGTSGGFYYSFWTDTPNSVTYTNGDAGKFSVSWKNNNGNHVGGKGWRTGSARTIKYSGSYKPNGNSYLAIYGWTRSPLIEYYIVESFGTYNPSTGATSKGQFTVDGSVYDLYTSTRTNAPSIEGTRTFTQFWSVRRTKRTSGSVNTGAHFAAWKKAGMNLGSHDYQILAVEGYKSSGSATMTVS,new,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A2H5AIX5,MGSIPSEKETIFRSKLPDIYVPDHLPLHSYCFQNLHQFSDRPCLIDGFTNKTLTYAEVELASKRVGAGLHRLGLRQGHVVMLLLPNSIEFVLSFIGASLLGAMSTTANPFYTSAEIHKQAAAAGAKIIVTESCHVSKLQGLEGISRIVVIDDAVRVPENVMHFSELESTDEAELPRIDVHPDDVVALPYSSGTTGLPKGVMLTHNGLVTSVAQQVDGENPNLHFSEDDVLLCVLPLFHIYSLNSVLLCGLRAGAAIVLMRKFEIVRLMELVEKYRVTIAPFVPPIVVEMVKNEAVDRYDLSSIRVVMSGAAPMGKELENKLREKLPNAKLGQGYGMTEAGPVLSMCLAFAKEPFEVKSGSCGTVVRNAELKIIDPETGFSLSRNQPGEICIRGNQIMKGYLNNPEATKQTIDEEGWLHTGDIGFVDDDDEIFIVDRLKELIKYKGFQVAPAELEAMLITHPNMADAAVVSIKDDSCGELPVAFIVRSNGSEITEDEIKKYISKQVVFYKRIHRVFFIEAIPKAPSGKILRKELRARLAAECPNGRQL,new,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
I3PB37,MPMETETNQGDLIFRSKLPDIYIPKHLPLHSYCFENISEFSSRPCLINGANNHIYTYADVELTSRKVAAGLNKLGIQQKDTIMILLPNSPEFVFAFMGASYLGAISTMANPLFTPAEVVKQAKASNAKLIITQACFVNKVKDYAFDNNLNVICIDSAPEGCIHFSELTQADEHDIPDVKIQSDDVVALPYSSGTTGLPKGVMLTHKGLVTSVAQQVDGENANLYMHSEDVLMCVLPLFHIYSLNSVLLCGLRVGAAILIMQKFDIVQFCELIEKYKVTIGPFVPPIVLAIAKSPVVDNYDLSSVRTVMSGAAPLGKELEDAVRIKFPNAKLGQGYGMTEAGPVLAMCLAFAKEPFDIKSGACGTVVRNAEMKIVDPDTGCSLPRNQPGEICIRGDQIMKGYLNDPAATTRTIDKEGWLHTGDIGYIDNDDELFIVDRLKELIKYKGFQVAPAELEALLLNHPNISDAAVVPMKDEQAGEVPVAFVVRSNGSDITEDEVKDFVSKQVIFYKRIKRVFFVETVPKSPSGKILRKDLRARLAAGVPN,new,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A2H5AIY4,MITIAESHPQIHHSPPDTTAPSTPPTATIFRSRLPDIVLSNHLPLHEYLFENTLTSPSPCIISASTGRSYSFAETHLLSRKTASFLSSRCGVRRGGVVMLLLHNCPEFVFSFLGSSMLGAVTTAANPFCTPQEIKKQLLASGATVIITQSAYASKISRDDDVEDLIVVTVSDNELERHAAPPEGCVSFSEVESADEDAVPDAVGVGPEDAVAMPFSSGTTGLPKGVVLTHKSMTSSVGQIVDGENPNLHLRKGEDVLLCVLPLFHIFSLNSVLLCGLRTGAAVVIMARFEMEGMLETIQRWGVSVAAVVPPLVLALAKNPLVEKYDMGTVRMVLSGAAPLGKELEAVLKGRLPQAVLGQGYGMTEAGPVISMSPGFAKQPTPVKSGSCGTVVRNAELKVMDPETGFSLGRNQPGEICVRGPQIMKGYLNDPEATSATIDVEGWLHTGDVGYVDDDDEVFIVDRVKELIKFKGFQVPPAELEALLLGHPSIADAAVIPQNDEVAGEVPVAFVVPSKSSDLTEEIVKEFISKQVVFYKRIHRVYFIHAIPKSPSGKILRKDLRAKVASFSS,new,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P52307,GRLLACLLLPGLAATDFTATVLHTNDVHGRFEQITASGTRCTKQAAEAQQCVGGIARQKTVVSQAAASGANVLFLNAGDYYQGSIWYYVLGAPIVAEAVNYLAHDAMALGNHEFDRGAEGLVPLLTESRVPIVGCNVDFSEEPTLKPLQPKPSVVVERAGIKIGLIGYTTMNTTYLSSPGKVRFTDEAECIQREAQRLRREEGVQVIIAVGHSGVPRDLEICERVPEVSLVVGGHTHTFLYSGPTENGRVSGDKPQGPYPIVVDRAADSRCLVVQDFYMGKYMGNISITWNQRGEVVRWSGQPVLLDRSIPEDPDGIALLDRYRDRVAQSQASVVAESKVLLDGDKNRCRLDECNLGNLVMDAFLKKMASLPSPQASWTRVAAVIANAGGIRASIDEQSTGGRITFEDVVNVLPFGNTLVEMNVTGAQLKGLLEHGVHRHDVKGYVPPAELVLAAGLRVVYDIQREPYDRVVEAHILCTECRVPRYEPLEYARQYRIAALSYIVHGGDNFDFSFVKPKDMYDTGFQDAEIVMKYMNSTSPITTALDGRVTFLKTNQASDACLNLASPFLVLLVLVVFYHL,new,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A7ZI07,MLKRLLSKLTGNRQQIEHHLKNQYQVEENGLSFPLSLVDDSQLWALASWLEQLAEEDYLISLTDRWLLSWDALYRLLEDEEHASSLPLIGVPDVLPLRASLSSRGALSDSDFRVWIAEWATLPARKPIRFSRTGAILTHENQQYLLSRENWALLQATEQLSAQKNQTPGETTNQLGWAAIRKCAKQAAAKFDDYLEKTHVVKPTSLSLRLRKATVADTAVIEIEPHFEDQPANWLGSFDKNSQVHDSYRIPGENGELSHVIIPPEVKEVLNSIHSIPSRRVAGSEALSFVRNPYTFLGEDAASVIAPEEHEQALFDARIFFHHFRLIPQLNAENKIAEVTLVLEPVSPVPQPEITFGFSAPRELDKFIQQLGISVAAQMPAGSWQGYELELSQFTEQQWHDCQALLTRWQQEIEGKEFSDVLDIAKYGDRVIGIGEFEKISSPWLTKAQSENWLPDDIDFSAFSVETLSGWQPENLHHFDELQERITQAEAVGETHITAPWNDSQLPLDAAKTFSKNWEKQQSTANESQGNVADKTARAVLKIEQNIEETAYIKQRRNSLLNARHAEPEIPLSLKEHIRLKDHQREGVAWLQQLFLRSPEETAGCLLADDMGLGKTLQILSFLVWFIEKFPQEPPSLIVAPVSLLDNWERELDNFFYTAGIPVLKLYGETIKAVKYPKQAIPAHLQSQGIKNLLKPGWQGEAKIILTTYETLRDQEFSLARQPWSIMVCDEAQKIKNPAALITHAANAVQARFKVACTGTPVENTLVDLWSLFDFAQPGLLGALNEFGKHYVRPIENEDGRDTERLESLRALIEPQTLRRTKEEVARDLPQKIEVESCKQLTLSGVQKQLYLSSVANWQQQQALSEGMQQAGTGMLGLLHRL

In [90]:
split = 'new'
label_mask = labels.query('split == @split').sum(axis=0)>0
label_mask = list(label_mask.index) #list(unseen_terms&set(labels.query('split == @split').columns)) 

mAP_micro = BinaryAUPRC(device='cpu')
mAP_macro = MultilabelAUPRC(device='cpu',
                            num_labels=len(label_mask))

probabilities = torch.sigmoid(torch.tensor(logits.query('split == @split')[label_mask].values))
y = torch.tensor(labels.query('split == @split')[label_mask].values)


print(mAP_micro.update(probabilities.flatten(), y.flatten()).compute(),mAP_macro.update(probabilities, y).compute())

tensor(0.0212) tensor(0.0431)


In [89]:
(labels.query('split == @split')[label_mask].sum(axis=1)>0).sum()

987

In [87]:
0.024638491995179892*44000

1084.0936477879152

In [15]:


logits_unseen = pd.read_parquet("../outputs/results/unseen_zero_shot_logits.parquet")
labels_unseen = pd.read_parquet("../outputs/results/unseen_zero_shot_labels.parquet")
mAP_micro = BinaryAUPRC(device='cpu')
mAP_macro = MultilabelAUPRC(device='cpu',
                            num_labels=labels_unseen.shape[-1])

probabilities = torch.sigmoid(torch.tensor(logits_unseen.values))
y = torch.tensor(labels_unseen.values)

print(mAP_micro.update(probabilities.flatten(), y.flatten()).compute(),mAP_macro.update(probabilities, y).compute())

tensor(0.1921) tensor(0.2617)


In [28]:

mask = unseen_seq_ids&set(labels.index.get_level_values(0))
cols = unseen_terms&set(labels.columns)

mAP_micro = BinaryAUPRC(device='cpu')
mAP_macro = MultilabelAUPRC(device='cpu',
                            num_labels=len(cols))

probabilities = torch.sigmoid(torch.tensor(logits.loc[mask,cols].values))
y = torch.tensor(labels.loc[mask,cols].values)
print(mAP_micro.update(probabilities.flatten(), y.flatten()).compute(),mAP_macro.update(probabilities, y).compute())

/tmp/ipykernel_4119746/2109383899.py:8: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  probabilities = torch.sigmoid(torch.tensor(logits.loc[mask,cols].values))
/tmp/ipykernel_4119746/2109383899.py:9: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  y = torch.tensor(labels.loc[mask,cols].values)


tensor(0.1861) tensor(0.2622)


In [74]:
t=(labels.loc[mask].sum(axis=0)>0)
t=set(t[t].index)

mask = unseen_seq_ids&set(labels.index.get_level_values(0))
cols = set(labels.columns)&t

mAP_micro = BinaryAUPRC(device='cpu')
mAP_macro = MultilabelAUPRC(device='cpu',
                            num_labels=len(cols))

probabilities = torch.sigmoid(torch.tensor(logits.loc[mask,cols].values))
y = torch.tensor(labels.loc[mask,cols].values)
print(mAP_micro.update(probabilities.flatten(), y.flatten()).compute(),mAP_macro.update(probabilities, y).compute())

/tmp/ipykernel_4119746/3361736342.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  t=(labels.loc[mask].sum(axis=0)>0)
/tmp/ipykernel_4119746/3361736342.py:11: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  probabilities = torch.sigmoid(torch.tensor(logits.loc[mask,cols].values))
/tmp/ipykernel_4119746/3361736342.py:12: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  y = torch.tensor(labels.loc[mask,cols].values)


tensor(0.1893) tensor(0.2676)


In [75]:
logits.loc[mask,cols]

/tmp/ipykernel_4119746/330024805.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  logits.loc[mask,cols]


,,,GO:0120293,GO:0106423,GO:0062146,GO:0140900,GO:0140471,GO:0120284,GO:0140416,GO:0140664,GO:0106408,GO:0062202,...,GO:0140981,GO:0140445,GO:0120228,GO:0160013,GO:0141014,GO:0140722,GO:0106219,GO:0140460,GO:0106429,GO:0160044
,sequence,split,,,,,,,,,,,,,,,,,,,,,
A5F7V7,MSVPTQITFFEFLTPLVASGQKTITIRDKSESHYVPGTRVEVFTLETQRKVCEIDILAVEPLKFDEINEFHAEQEAIELPKLKALIQEIYPNIDELYVITYQLAKSSSARIMVK,new,-5.078125,-3.626953,-2.724609,-14.843750,-4.925781,-2.617188,-1.403320,-2.847656,-1.186523,-3.082031,...,-6.269531,-5.628906,-7.277344,-4.078125,-2.275391,-5.261719,-4.042969,-4.117188,-6.175781,-13.640625
O86411,STGSLGLDLALGVGGLPRGRVVKIFGPESSGKTTLCLEAIAQCQKNGGICAFIDAEHAFDPVYARKLGIKVEELYLSQPDTGEQALEICDTLVRSGGVDMVVVDSVAALVPKAEIEGEMGDSHVGLQARLMSQALRKLTGHIKRTNTLVVFINQIRMKIGVMFGSPETTTGGNALKFYASVRLDIRRTGQIKKGDDVIGNETKVKVIKNKVAPPFRQAEFDILYGEGVSWEGELIDL,new,-5.289062,-5.414062,-6.007812,-16.125000,-2.875000,-4.523438,-3.259766,-1.560547,-2.523438,-5.097656,...,-15.945312,-4.105469,-8.281250,-6.171875,-5.542969,-13.460938,-3.443359,-2.894531,-7.835938,-15.945312
Q8XBX8,MLSSLNVLQSSFRGKTALSNSTLLQKVSFAGKEYPLEPIDEKTPILFQWFEARPERYEKGEVPILNTKEHPYLSNIINAAKIENERIIGVLVDGNFTYEQKKEFLSLENEYQNIKIIYRADVDFSMYDKKLSDIYLENIHKQESYPASERDNYLLGLLREELKNIPEGKDSLIESYAEKREHTWFDFFRNLAMLKAGSLFTETGKTGCHNISPCSGCIYLDADMIITDKLGVLYAPDGIAVHVDCNDEIKSLENGAIVVNRSNHPALLAGLDIMKSKVDAHPYYDGLGKGIKRHFNYSSLHDYNAFCDFIEFKHENIIPNTSMYTCSSW,new,-5.257812,-3.601562,-2.550781,-15.539062,-5.332031,-3.882812,-3.160156,-3.039062,-3.041016,-3.396484,...,-6.023438,-2.566406,-9.187500,-2.820312,-3.375000,-8.335938,-2.146484,-2.173828,-6.070312,-16.125000
A0A3B1EFP9,MGASHSTVSDPHRFAVAITGSDDDVRSLQAHPTKSYDYIVVGGGTAGCVLASRLSEDSRVNVLLVEAGYSNHGVTNSIIPLAFPMLMKSKYDWNYETVGMAGINGRTSYWPRGRLLGGTSSINGSMYHRCAPEDFSAWVEEGAKGWEYENMKPYFRKAEGYNPHPDHPNIDPALHGTVGPAKVTHGPIAFLSQPITKDILQSSINVGIRQVHDFNTDVGPTGVGLFARNVFPNGTRVSAATGYLTPSVLARPNLTVAVECMAEKIVLSSDEKVPRAKGLIFSTSRDGQRFYVPASKELILSSGVIGTPQVLMLSGIGPAAELAKHNIPVVRDLPVGEYLQDHFSPGPMLIRAKKGTWDFILRPLGGLLAMVKWFFFGKGPLSSLSVQVACFVRSDDKTFAHPQLPMHDGAKKYQVKDCCSGPTAPDIELFFAPFLAPPLGMKYWPFPGLTSGMLLLKPASYGTVKLASRNAWDYPLIDPNYLSHESDIALSIMGMRLLARIARTKPFADSLDLPEDSDDKTSVFWPTDCNPDTVSDDDLEAWARLHGQSTGHPTSSARMGGSPSTSVVDSKLKVHGVEGLRVCDASCFPTQVSGHPAAVVVAVAERAADLIKGVA,new,-6.242188,-5.460938,-3.912109,-16.125000,-16.125000,-4.925781,-6.128906,-3.292969,-5.519531,-3.406250,...,-2.900391,-8.828125,-16.125000,-3.511719,-4.667969,-6.339844,-5.589844,-5.601562,-4.882812,-16.125000
W6R4H4,MLTKSVTSILQGITLAAKEFENNEAGARESLIAHSRALISALEVPSEFIQHTFWSQPALSAIIRLAADVNMFQHLKDAAGKGIDCEALSMKTGVDASLLSRLARHLVAMNVITFQNGAFHGTDLSDSLAAENYQHSIRFCHDVSRPSFNEFPEFFKSNGYKTPTLSGTDGPFQAAHKTELTFLQWLVNTHPYLQYFHSYMSVYRAGKQNWCDTGFYPVSERLLSGFDASVSDVVLVDVGGGRGHDLETFASKFSPLPGRLVLQDREQTIASMPADESRQFEATAHNIFTPQPVKYARAYYMHSVPHGFGDEDAIKIMANLVPALAKGYSRVLLNEIVVSEENPILAATNMDMIMLAHLAVKERTEAEWRYIFTQAGLKVVNIYSYPGVAESLIEAELA,new,-6.085938,-4.105469,-3.187500,-16.125000,-11.359375,-4.269531,-5.734375,-4.007812,-4.835938,-3.160156,...,-4.703125,-7.164062,-10.226562,-4.421875,-4.183594,-4.195312,-4.316406,-4.125000,-4.406250,-16.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P86233,GAVVGIDLGTTNSCVAVMEGKVLENAEGARTTPSVVAFTADGERQAVTNPNNTFYATKRRYDDPEVQKNVPFKIVRASNGDAWVEAHGKLYSPSQIGAFVLMKMKNAVITVPAYFNDSQRQATKDAGQISGLNVLRSTNGDTFLGGEDFDQALLRETGVDLTKDNMALQRAQFEGIVTDLIKRAMQDAEVSKSDIGEVILVGGMTRVQQTVQDLFGRAPSKSQVFSTAADGQTQVEIKEMAGDNKLLGQFTLIGIPPAPRGVPQIEVTFDIDANGIVHVSAKEQQIVIQSSGGLSKDDIENMVKYAEEDRRERVEAVNMAEGIIHDTETKMEEFKDQLPADECNKMRELLARKDSETGENIRLFEMAYKK,new,-5.816406,-2.927734,-2.443359,-10.875000,-3.822266,-3.511719,-3.341797,-3.437500,-2.130859,-3.433594,...,-4.500000,-5.863281,-8.640625,-3.697266,-3.658203,-3.951172,-3.232422,-2.453125,-4.519531,-9.945312
Q29068,NAAKTIADIIRTCLGPKSMMKMLLDPMGGIVMTNDGNAILREIQVQHPAAKSMIEISRTQDEEVGDGTTSVIILAGEMLSVA,new,-10.203125,-12.070312,-16.125000,-16.125000,-16.125000,-13.414062,-7.210938,-8.906250,-13.054688,-12.476562,...,-16.125000,-11.765625,-16.125000,-16.125000,-8.554688,

In [27]:
m.mean()

tensor(0.2622)

In [22]:
labels.loc[mask,cols].shape

/tmp/ipykernel_4119746/662757340.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  labels.loc[mask,cols].shape


(894, 227)

In [52]:
set(labels_unseen.columns) - set(labels.columns)

{'GO:0106259'}

In [213]:
m[y.sum(axis=0)!=0].mean()

tensor(0.2640)

In [17]:
m.mean()

tensor(0.0843)